# Getting ZECMIP File Paths

- Getting all the file paths for zecmip takes a while to laod with glob. Savign all these files in a csv files to improve speed.

In [10]:
from glob import glob
import pandas as pd
import sys, os
import numpy as np
sys.path.append(os.path.join(os.getcwd(), 'Documents/PhD'))
import constants
import json
from pprint import pprint
from typing import List, Dict
import copy
from collections import defaultdict
from pathlib import Path

In [11]:
constants.DECK_DIR, constants.ZECMIP_DIR

('/g/data/oi10/replicas/CMIP6/CMIP', '/g/data/oi10/replicas/CMIP6/C4MIP')

In [12]:
def sort_into_ensemble(path_list: List[str]) -> Dict[str, Dict[str, str]]:
    """
    Sorts a list of file paths into an ensemble dictionary based on the model and its associated ensemble members.

    Args:
        path_list (List[str]): A list of file paths containing information about models, ensemble members, and frequencies.

    Returns:
        Dict[str, Dict[str, str]]: A dictionary where each model maps to a dictionary containing ensemble members as keys
        and their corresponding file paths as values.
    """
    # Create a defaultdict with nested dictionaries as the default factory
    stor_obj = defaultdict(dict)

    for path in path_list:
        # Extract model, ensemble_member, and frequency from the file path
        model, ensemble_member, _ = path.split('/')[8], path.split('/')[10], path.split('/')[11]

        # Find all ensemble members for the current model
        ensemble_members = [path2.split('/')[10] for path2 in path_list if ensemble_member not in path2 and model in path2]

        # Update the ensemble dictionary for the current model
        # Using the update method and dict comprehension to add the current ensemble member and all others found
        stor_obj[model].update({ensemble_member: path, **{member: path2 for member, path2 in zip(ensemble_members, path_list) if member != ensemble_member and model in path2}})

    # Convert the defaultdict to a regular dictionary and return the result
    return dict(stor_obj)


In [13]:
def create_ensemble_summary(base_path_sorted_ensemble):
    """
    Creates an ensemble summary dictionary based on the given sorted ensemble data.

    The function takes the `base_path_sorted_ensemble`, which is a dictionary representing the sorted ensemble data.
    It contains information about models and their associated ensemble members' file paths.
    The `base_path_sorted_ensemble` dictionary has the following structure:
    {
        'model_1': {
            'ensemble_member_1': 'file_path_for_ensemble_member_1',
            'ensemble_member_2': 'file_path_for_ensemble_member_2',
            ...
        },
        'model_2': {
            'ensemble_member_1': 'file_path_for_ensemble_member_1',
            'ensemble_member_2': 'file_path_for_ensemble_member_2',
            ...
        },
        ...
    }

    The function creates an ensemble summary dictionary, `stor_obj_to_return`, where each model maps to a dictionary.
    The inner dictionaries contain information about ensemble members associated with that model.
    If an ensemble member is missing for a specific model, it is not included in the inner dictionary.

    Args:
        base_path_sorted_ensemble (dict): A dictionary representing the sorted ensemble data.

    Returns:
        dict: An ensemble summary dictionary where each model maps to a dictionary containing ensemble members as keys
        and their corresponding file paths as values.
    """
    stor_obj_to_return = {}

    for model, values in base_path_sorted_ensemble['1000pgc'].items():
        # Use dictionary comprehension to create the model_dict containing ensemble members for the current model
        model_dict = {exp: base_path_sorted_ensemble[exp][model] for exp in base_path_sorted_ensemble if exp != '1000pgc' and model in base_path_sorted_ensemble[exp]}
        
        # Add the ensemble member values for the current model from '1000pgc'
        model_dict['1000pgc'] = values

        # Add the model_dict to the stor_obj_to_return
        stor_obj_to_return[model] = model_dict

    return stor_obj_to_return


In [14]:
%%time
# Sample file name: tas_Amon_IPSL-CM5A2-INCA_piControl_r1i1p1f1_gr_185001-209912.nc
fpaths_1000PgC = glob(constants.ZECMIP_DIR + '/**/*tas_*Amon*brch-1000PgC*.nc', recursive=True)

CPU times: user 636 ms, sys: 3.45 s, total: 4.09 s
Wall time: 12 s


In [15]:
# Needs _piControl or else will also get esm-piControl
fpaths_picontrol = glob(constants.DECK_DIR + '/**/*tas_*Amon*_piControl*.nc', recursive=True)

In [16]:
fpaths_esm_picontrol = glob('/g/data/oi10/replicas/CMIP6/CMIP' + '/**/*tas_*Amon*esm-piControl*.nc', recursive=True)

In [17]:
fpaths_picontrol[:3]

['/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_314801-319612.nc',
 '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_246901-249512.nc',
 '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_236901-246812.nc']

In [18]:
fpaths_1pct = glob(constants.DECK_DIR + '/**/*tas_*Amon*1pctCO2*.nc', recursive=True)

In [19]:
fpaths_1pct.extend([
    # '/g/data/oi10/replicas/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/1pctCO2/r1i1p1f1/Amon/tas/gn/v20180905',
    '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/1pctCO2/r1i1p1f1/Amon/tas/gn/v20191115/'
])

fpaths_picontrol.extend([
    '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl/r1i1p1f1/Amon/tas/gn/v20210316/'
])

fpaths_1000PgC.extend([
    '/g/data/fs38/publications/CMIP6/C4MIP/CSIRO/ACCESS-ESM1-5/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191206/',
])

fpaths_esm_picontrol.extend([
    '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/esm-piControl/r1i1p1f1/Amon/tas/gn/v20191115/'
    # '/g/data/w40/ab2313/PhD/zecmip/original/esm_picontrol/GISS' 
])

In [20]:
fpaths_1pct[:3]

['/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/1pctCO2/r1i1p1f1/Amon/tas/gr1/v20200226/tas_Amon_INM-CM5-0_1pctCO2_r1i1p1f1_gr1_188401-196212.nc',
 '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/1pctCO2/r1i1p1f1/Amon/tas/gr1/v20200226/tas_Amon_INM-CM5-0_1pctCO2_r1i1p1f1_gr1_196301-199912.nc',
 '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/1pctCO2/r1i1p1f1/Amon/tas/gr1/v20200226/tas_Amon_INM-CM5-0_1pctCO2_r1i1p1f1_gr1_185001-188312.nc']

In [21]:
base_paths = {
    'picontrol': np.sort(np.unique(list(map(os.path.dirname, fpaths_picontrol)))),
    'esm-piControl':np.sort(np.unique(list(map(os.path.dirname, fpaths_esm_picontrol)))),
    '1000pgc':np.sort(np.unique(list(map(os.path.dirname, fpaths_1000PgC)))),
    '1pct':np.sort(np.unique(list(map(os.path.dirname, fpaths_1pct))))}

In [22]:
base_path_sorted_ensemble = {key: sort_into_ensemble(base_path_list) for key, base_path_list in base_paths.items()}

In [23]:
pprint(base_path_sorted_ensemble['1000pgc'])

{'ACCESS-ESM1-5': {'r1i1p1f1': '/g/data/fs38/publications/CMIP6/C4MIP/CSIRO/ACCESS-ESM1-5/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191206'},
 'CESM2': {'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/C4MIP/NCAR/CESM2/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191119'},
 'CanESM5': {'r1i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
             'r2i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
             'r3i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r2i1p2f1/Amon/tas/gn/v20190429',
             'r4i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r3i1p2f1/Amon/tas/gn/v20190429',
             'r5i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r5i1p2f1/Amon/tas/gn/v20190429'},
 'GFDL-ESM4': {'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/C4MIP/NOAA-GFDL/GFDL-ESM4/e

In [24]:
base_path_sorted_ensemble_model = create_ensemble_summary(base_path_sorted_ensemble)

In [25]:
pprint(base_path_sorted_ensemble_model['CanESM5'])

{'1000pgc': {'r1i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
             'r2i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
             'r3i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r2i1p2f1/Amon/tas/gn/v20190429',
             'r4i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r3i1p2f1/Amon/tas/gn/v20190429',
             'r5i1p2f1': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r5i1p2f1/Amon/tas/gn/v20190429'},
 '1pct': {'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/CMIP/CCCma/CanESM5/1pctCO2/r1i1p1f1/Amon/tas/gn/v20190429',
          'r1i1p2f1': '/g/data/oi10/replicas/CMIP6/CMIP/CCCma/CanESM5/1pctCO2/r1i1p2f1/Amon/tas/gn/v20190429',
          'r2i1p1f1': '/g/data/oi10/replicas/CMIP6/CMIP/CCCma/CanESM5/1pctCO2/r2i1p1f1/Amon/tas/gn/v20190429',
          'r2i1p2f1':

In [26]:
pprint(base_path_sorted_ensemble_model['GISS-E2-1-G-CC'])

{'1000pgc': {'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/C4MIP/NASA-GISS/GISS-E2-1-G-CC/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191202'},
 'picontrol': {'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G-CC/piControl/r1i1p1f1/Amon/tas/gn/v20190815'}}


In [27]:
base_path_sorted_ensemble_model['GISS-E2-1-G-CC'].update(
    **{'esm-piControl' :{'r1i1p1f1':'/g/data/w40/ab2313/PhD/zecmip/original/esm_picontrol/GISS'}, 
       '1pct':{'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/1pctCO2/r1i1p1f1/Amon/tas/gn/v20180905'}
    })

In [33]:
base_path_sorted_ensemble_model['UKESM1-0-LL'].update(
    **{'esm-piControl' :{'r1i1p1f1':'/g/data/w40/ab2313/PhD/zecmip/original/esm_picontrol/UKESM1-0-LL'}, 
    })

In [35]:
base_path_sorted_ensemble_model['UKESM1-0-LL']

{'picontrol': {'r1i1p1f2': '/g/data/oi10/replicas/CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f2/Amon/tas/gn/v20200828'},
 'esm-piControl': {'r1i1p1f1': '/g/data/w40/ab2313/PhD/zecmip/original/esm_picontrol/UKESM1-0-LL'},
 '1pct': {'r1i1p1f2': '/g/data/oi10/replicas/CMIP6/CMIP/MOHC/UKESM1-0-LL/1pctCO2/r1i1p1f2/Amon/tas/gn/v20190406',
  'r2i1p1f2': '/g/data/oi10/replicas/CMIP6/CMIP/MOHC/UKESM1-0-LL/1pctCO2/r2i1p1f2/Amon/tas/gn/v20190603',
  'r3i1p1f2': '/g/data/oi10/replicas/CMIP6/CMIP/MOHC/UKESM1-0-LL/1pctCO2/r3i1p1f2/Amon/tas/gn/v20190604',
  'r4i1p1f2': '/g/data/oi10/replicas/CMIP6/CMIP/MOHC/UKESM1-0-LL/1pctCO2/r4i1p1f2/Amon/tas/gn/v20190604'},
 '1000pgc': {'r1i1p1f2': '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200210',
  'r2i1p1f2': '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r2i1p1f2/Amon/tas/gn/v20200106',
  'r3i1p1f2': '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r3i1

In [36]:
pprint(base_path_sorted_ensemble_model)

{'ACCESS-ESM1-5': {'1000pgc': {'r1i1p1f1': '/g/data/fs38/publications/CMIP6/C4MIP/CSIRO/ACCESS-ESM1-5/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191206'},
                   '1pct': {'r1i1p1f1': '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/1pctCO2/r1i1p1f1/Amon/tas/gn/v20191115'},
                   'esm-piControl': {'r1i1p1f1': '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/esm-piControl/r1i1p1f1/Amon/tas/gn/v20191115'},
                   'picontrol': {'r1i1p1f1': '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl/r1i1p1f1/Amon/tas/gn/v20210316'}},
 'CESM2': {'1000pgc': {'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/C4MIP/NCAR/CESM2/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191119'},
           '1pct': {'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Amon/tas/gn/v20190425'},
           'esm-piControl': {'r1i1p1f1': '/g/data/oi10/replicas/CMIP6/CMIP/NCAR/CESM2/esm-piControl/r1i1p1f1/Amon/tas/gn/v20190723'},
         

In [39]:
with open(Path.cwd() / 'Documents' / 'PhD' / 'data' /'zecmip_experiment_paths_ensemble_sorted.json', 'w') as fp:
    json.dump(base_path_sorted_ensemble_model, fp)